In [1]:
import json
from pathlib import Path

from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd

import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBoxesOnImage

from utils.config import *
from utils.utils import *

In [ ]:
with open('./path/train_imgs_path.txt', 'r', encoding='utf-8') as f:
    train_imgs_path = f.read().splitlines()
with open('./path/valid_imgs_path.txt', 'r', encoding='utf-8') as f:
    valid_imgs_path = f.read().splitlines()
with open('./path/test_imgs_path.txt', 'r', encoding='utf-8') as f:
    test_imgs_path = f.read().splitlines()

with open('./path/train_labels_path.txt', 'r', encoding='utf-8') as f:
    train_labels_path = f.read().splitlines()
with open('./path/valid_labels_path.txt', 'r', encoding='utf-8') as f:
    valid_labels_path = f.read().splitlines()
with open('./path/test_labels_path.txt', 'r', encoding='utf-8') as f:
    test_labels_path = f.read().splitlines()

class_encoding = pd.read_csv('./class_encoding.csv')
class_nums = dict(class_encoding[['encoding_name', 'class_num']].values)
train_encoding = dict(class_encoding[['car_person', 'encoding_name']].values)
test_encoding = dict(class_encoding[['cityscapes', 'encoding_name']].values)

In [ ]:
train_seq = iaa.Sequential([iaa.size.Resize((RESIZE_TARGET, RESIZE_TARGET))])
test_seq = iaa.Sequential(
    [
        iaa.CropToAspectRatio(TRAIN_ASPECT_RATIO, position='right-bottom'),
        iaa.size.Resize((RESIZE_TARGET, RESIZE_TARGET))
    ]
)

In [9]:
for i in tqdm(range(len(train_imgs_path))):
    img_path = train_imgs_path[i]
    label_path = train_labels_path[i]
    img_path_obj = Path(img_path)
    label_path_obj = Path(label_path)
    dst_img = (Path(*img_path_obj.parts[:3]) / 'images').joinpath(*img_path_obj.parts[3:]).with_suffix('.jpg')
    dst_label = (Path(*label_path_obj.parts[:3]) / 'labels').joinpath(*label_path_obj.parts[3:]).with_suffix('.txt')
    
    img = read_kor_path_img(img_path)
    if img is not None:
        with open(label_path, 'r', encoding='utf-8') as j:
            label = json.load(j)
            label = label['annotations']
        bbs = label2bbs(label, train_encoding, 'car_person')
        bbs_on_img = BoundingBoxesOnImage(bbs, shape=img.shape)
        resize_img, resize_bbs_on_img = train_seq(image=img, bounding_boxes=bbs_on_img)
        resize_bbs_on_img = resize_bbs_on_img.clip_out_of_image()
        resize_yolo_instances = [
        bb2yolo(bb, resize_bbs_on_img.width, resize_bbs_on_img.height, class_nums) for bb in resize_bbs_on_img
        ]
        
        dst_img.parent.mkdir(parents=True, exist_ok=True)
        dst_label.parent.mkdir(parents=True, exist_ok=True)
        Image.fromarray(resize_img).save(str(dst_img), format='JPEG', quality=95)
        with open(dst_label, 'w', encoding='utf-8') as f:
            for instance in resize_yolo_instances:
                f.writelines(' '.join([str(x) for x in instance]) + '\n')

  0%|          | 0/406514 [00:00<?, ?it/s]

In [12]:
for i in tqdm(range(len(valid_imgs_path))):
    img_path = valid_imgs_path[i]
    label_path = valid_labels_path[i]
    img_path_obj = Path(img_path)
    label_path_obj = Path(label_path)
    dst_img = (Path(*img_path_obj.parts[:3]) / 'images').joinpath(*img_path_obj.parts[3:]).with_suffix('.jpg')
    dst_label = (Path(*label_path_obj.parts[:3]) / 'labels').joinpath(*label_path_obj.parts[3:]).with_suffix('.txt')
    
    img = read_kor_path_img(img_path)
    if img is not None:
        with open(label_path, 'r', encoding='utf-8') as j:
            label = json.load(j)
            label = label['annotations']
        bbs = label2bbs(label, train_encoding, 'car_person')
        bbs_on_img = BoundingBoxesOnImage(bbs, shape=img.shape)
        resize_img, resize_bbs_on_img = train_seq(image=img, bounding_boxes=bbs_on_img)
        resize_bbs_on_img = resize_bbs_on_img.clip_out_of_image()
        resize_yolo_instances = [
        bb2yolo(bb, resize_bbs_on_img.width, resize_bbs_on_img.height, class_nums) for bb in resize_bbs_on_img
        ]
        
        dst_img.parent.mkdir(parents=True, exist_ok=True)
        dst_label.parent.mkdir(parents=True, exist_ok=True)
        Image.fromarray(resize_img).save(str(dst_img), format='JPEG', quality=95)
        with open(dst_label, 'w', encoding='utf-8') as f:
            for instance in resize_yolo_instances:
                f.writelines(' '.join([str(x) for x in instance]) + '\n')

  0%|          | 0/112771 [00:00<?, ?it/s]

In [32]:
for i in tqdm(range(len(test_imgs_path))):
    img_path = test_imgs_path[i]
    label_path = test_labels_path[i]
    img_path_obj = Path(img_path)
    label_path_obj = Path(label_path)
    label_fname = label_path_obj.stem.replace('gtFine_polygons', 'leftImg8bit') + '.txt'
    dst_img = (Path(*img_path_obj.parts[:3]) / 'images').joinpath(*img_path_obj.parts[3:]).with_suffix('.jpg')
    dst_label = (Path(*label_path_obj.parts[:3]) / 'labels' / 'leftImg8bit').joinpath(*label_path_obj.parts[4:-1]) / label_fname

    img = read_kor_path_img(img_path)
    if img is not None:
        with open(label_path, 'r', encoding='utf-8') as j:
            label = json.load(j)
        bbs = label2bbs(label, test_encoding, 'cityscapes')
        bbs_on_img = BoundingBoxesOnImage(bbs, shape=img.shape)
        resize_img, resize_bbs_on_img = test_seq(image=img, bounding_boxes=bbs_on_img)
        resize_bbs_on_img = resize_bbs_on_img.clip_out_of_image()
        resize_yolo_instances = [
        bb2yolo(bb, resize_bbs_on_img.width, resize_bbs_on_img.height, class_nums) for bb in resize_bbs_on_img
        ]
        
        dst_img.parent.mkdir(parents=True, exist_ok=True)
        dst_label.parent.mkdir(parents=True, exist_ok=True)
        Image.fromarray(resize_img).save(str(dst_img), format='JPEG', quality=95)
        with open(dst_label, 'w', encoding='utf-8') as f:
            for instance in resize_yolo_instances:
                f.writelines(' '.join([str(x) for x in instance]) + '\n')

  0%|          | 0/500 [00:00<?, ?it/s]

In [7]:
root_resize_train_img = Path(*Path(train_imgs_path[0]).parts[:3]) / 'images' / 'Training'
root_resize_train_label = Path(*Path(train_labels_path[0]).parts[:3]) / 'labels' / 'Training'
root_resize_valid_img = Path(*Path(valid_imgs_path[0]).parts[:3]) / 'images' / 'Validation'
root_resize_valid_label = Path(*Path(valid_labels_path[0]).parts[:3]) / 'labels' / 'Validation'
root_resize_test_img = Path(*Path(test_imgs_path[0]).parts[:3]) / 'images'
root_resize_test_label = Path(*Path(test_labels_path[0]).parts[:3]) / 'labels'

resize_train_imgs_path = list(root_resize_train_img.glob('**/*.jpg'))
resize_train_labels_path = list(root_resize_train_label.glob('**/*.txt'))
resize_valid_imgs_path = list(root_resize_valid_img.glob('**/*.jpg'))
resize_valid_labels_path = list(root_resize_valid_label.glob('**/*.txt'))
resize_test_imgs_path = list(root_resize_test_img.glob('**/*.jpg'))
resize_test_labels_path = list(root_resize_test_label.glob('**/*.txt'))

In [11]:
with open('../path/resize_train_imgs_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_train_imgs_path])
with open('../path/resize_valid_imgs_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_valid_imgs_path])
with open('../path/resize_test_imgs_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_test_imgs_path])

with open('../path/resize_train_labels_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_train_labels_path])
with open('../path/resize_valid_labels_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_valid_labels_path])
with open('../path/resize_test_labels_path.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(x)+'\n' for x in resize_test_labels_path])